## Your Challenge

Modify the block above to keep track of the total time steps, and use that as a metric as to how good our Q-learning system is. You might want to increase the number of simulated trips, and remove the sleep() calls to allow you to run over more samples.

Now, try experimenting with the hyperparameters. How low can the number of epochs go before our model starts to suffer? Can you come up with better learning rates, discount factors, or exploration factors to make the training more efficient? The exploration vs. exploitation rate in particular is interesting to experiment with.

In [2]:
import numpy as np
import gym # importa a biblioteca do gym
import random

streets = gym.make("Taxi-v3", render_mode='ansi').env
streets.reset()

initial_state = streets.encode(2, 3, 2, 0) # define o estado inicial do taxi, onde o taxi está na posição 2,3, o passageiro na posição 2, e o destino na posição 0

streets.s = initial_state # define o estado inicial

q_table = np.zeros([streets.observation_space.n, streets.action_space.n]) # cria a q_table, que é uma matriz de zeros 
# com o número de linhas igual ao número de estados e o número de colunas igual ao número de ações possíveis

learning_rate = 0.1 # indica quanto o agente aprende a cada iteração
discount_factor = 0.6 # fator de desconto, indicando o quanto o agente desconta o valor futuro em relação ao valor presente
exploration = 0.1 # chance de escolher uma ação aleatória ao invés da melhor ação
epochs = 500 # número de iterações

for taxi_run in range(epochs):
    state = streets.reset()[0] # reseta o ambiente, para que o taxi comece em um estado aleatório
    done = False
    
    while not done:
        random_value = random.uniform(0, 1) # gera um valor aleatório entre 0 e 1, 
        # para decidir se o taxi vai explorar ou usar a melhor ação
        if (random_value < exploration): # condição para explorar
            action = streets.action_space.sample()  
        else: # escolhe a melhor ação
            action = np.argmax(q_table[state])
        
        next_state, reward, done, info, _ = streets.step(action)
        # a função step executa a ação escolhida e retorna o próximo estado.
        # leva em conta o nosso dicionário de recompensas.
        # (a recompensa (valor de q), se ja foi feita, e se o passageiro foi deixado no destino)

        prev_q = q_table[state, action] # valor de q do estado anterior
        next_max_q = np.max(q_table[next_state]) # valor de q do próximo estado
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q) 
        # calcula o novo valor de q para o estado atual 
        q_table[state, action] = new_q # atualiza o valor de q na q_table

        state = next_state

In [3]:
q_table[initial_state]
#o 0 representa a ação de mover para o sul, o 1 para o norte, o 2 para o leste, o 3 para o oeste, 
# o 4 para pegar o passageiro e o 5 para deixar o passageiro

array([-2.31369089, -2.30664087, -2.30943116, -2.31213942, -4.26997424,
       -7.2274761 ])

In [4]:
from IPython.display import clear_output
from time import sleep

# permite visualizar o taxi se movendo no ambiente
for tripnum in range(1, 11):
    state = streets.reset()[0] # adicionei o [0] para que o código funcione
   
    done = False
    trip_length = 0
    
    while not done and trip_length < 25:
        action = np.argmax(q_table[state])
        next_state, reward, done, info, _ = streets.step(action)
        clear_output(wait=True)
        print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        # print(streets.render(mode='ansi')) # quando fui rodar o código, há um erro dizendo que
        # o método render não aceita o argumento mode='ansi'
        print("\n"+streets.render())
        state = next_state
        trip_length += 1

    sleep(2)
    

Trip number 10 Step 24

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

